In [1]:
!conda install -y -c python3 cartopy
!pip install geoplot pygeohash geopy shapely geos geopandas descartes 

/bin/bash: conda: command not found
     |████████████████████████████████| 400 kB 11.7 MB/s 
     |████████████████████████████████| 994 kB 49.4 MB/s 
     |████████████████████████████████| 12.1 MB 127 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 1.3 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 
     |████████████████████████████████| 6.6 MB 13.7 MB/s 
     |████████████████████████████████| 219 kB 63.3 MB/s 
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6168 sha256=b38cf3a6cbd0020a73b7b2859c459cccf810bca8eeb3c68738f67130fe9acefa
  Stored in directory: /root/.cache/pip/wheels/e2/ed/86/be7141d743cfb4aebac76406187154cd78fe9e6535d872187a
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516274 sha256=01795b4bad9bfc82a7a16d5ec321b406deefe03fed4ce9616d40044e69a

In [2]:
from geopy import distance as geodist
import pygeohash as pgh
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import base64
from shapely.geometry import Point, Polygon

In [13]:
date_parser = lambda x: pd.to_datetime(x)

In [16]:
voyages = pd.read_csv('./voyages.csv', parse_dates=['begin_date', 'end_date'], date_parser=date_parser)
voyages.head()

,vessel,begin_date,end_date,begin_port_id,end_port_id
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0


Calculate time spent at port

In [62]:
time_spent = []

for i in range(len(voyages)):
    time_spent.append(voyages['end_date'].iloc[i] - voyages['begin_date'].iloc[i])

voyages = voyages.assign(time_spent = time_spent)
voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id,time_spent
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0,15 days 05:05:00
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0,26 days 21:17:00
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0,14 days 14:56:00
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0,14 days 01:59:00
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0,26 days 10:10:00
...,...,...,...,...,...,...
2055,176,2019-07-28 14:51:00,2019-08-01 08:49:00,64.0,136.0,3 days 17:58:00
2056,176,2019-08-02 04:28:00,2019-08-14 08:55:00,136.0,64.0,12 days 04:27:00
2057,176,2019-11-03 18:57:00,2019-11-20 02:30:00,64.0,56.0,16 days 07:33:00
2058,176,2019-11-21 02:29:00,2019-12-02 21:48:00,56.0,136.0,11 days 19:19:00


Calculate distance between two successive ports

In [109]:
distance = []
ports = pd.read_csv('./ports.csv', sep='\t')
ports = ports.reset_index(drop=True)
for i in range(len(voyages)):
  end_port = int(voyages['end_port_id'].iloc[i])
  begin_port = int(voyages['begin_port_id'].iloc[i])

  e_lat = ports.loc[ports['port'] == end_port].iloc[0][1]
  e_long = ports.loc[ports['port'] == end_port].iloc[0][2]
  b_lat = ports.loc[ports['port'] == begin_port].iloc[0][1]
  b_long = ports.loc[ports['port'] == begin_port].iloc[0][2]

  distance.append(geodist.distance((e_lat, e_long), (b_lat, b_long)).kilometers)

voyages = voyages.assign(distance = distance)
voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id,time_spent,distance
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0,15 days 05:05:00,7894.161113
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0,26 days 21:17:00,7894.161113
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0,14 days 14:56:00,7782.523134
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0,14 days 01:59:00,7782.523134
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0,26 days 10:10:00,11587.252161
...,...,...,...,...,...,...,...
2055,176,2019-07-28 14:51:00,2019-08-01 08:49:00,64.0,136.0,3 days 17:58:00,2307.565977
2056,176,2019-08-02 04:28:00,2019-08-14 08:55:00,136.0,64.0,12 days 04:27:00,2307.565977
2057,176,2019-11-03 18:57:00,2019-11-20 02:30:00,64.0,56.0,16 days 07:33:00,3643.242907
2058,176,2019-11-21 02:29:00,2019-12-02 21:48:00,56.0,136.0,11 days 19:19:00,2166.672542


Calculate efficiency

In [115]:
efficiency = []
for i in range(len(voyages)):
  time = (voyages['time_spent'].iloc[i]) / pd.offsets.Day()
  efficiency.append(voyages['distance'].iloc[i]/time)

voyages = voyages.assign(efficiency = efficiency)
voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id,time_spent,distance,efficiency
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0,15 days 05:05:00,7894.161113,518.949646
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0,26 days 21:17:00,7894.161113,293.607253
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0,14 days 14:56:00,7782.523134,532.239424
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0,14 days 01:59:00,7782.523134,552.632443
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0,26 days 10:10:00,11587.252161,438.518873
...,...,...,...,...,...,...,...,...
2055,176,2019-07-28 14:51:00,2019-08-01 08:49:00,64.0,136.0,3 days 17:58:00,2307.565977,615.578920
2056,176,2019-08-02 04:28:00,2019-08-14 08:55:00,136.0,64.0,12 days 04:27:00,2307.565977,189.371118
2057,176,2019-11-03 18:57:00,2019-11-20 02:30:00,64.0,56.0,16 days 07:33:00,3643.242907,223.312041
2058,176,2019-11-21 02:29:00,2019-12-02 21:48:00,56.0,136.0,11 days 19:19:00,2166.672542,183.540706


In [191]:
heading = []
tracking = pd.read_csv('./tracking.csv', sep='\t', parse_dates=['datetime'], date_parser=date_parser)

In [192]:
tracking = tracking.sort_values(['vessel', 'datetime'])
tracking

,vessel,datetime,lat,long,heading,speed,draft
629483,1,2019-01-01 01:53:00,29.743000,-93.869500,214.0,NaN,9.6
629484,1,2019-01-01 02:47:00,29.743000,-93.869500,214.0,0.0,9.6
629485,1,2019-01-01 05:47:00,29.743000,-93.869500,214.0,0.0,9.6
629486,1,2019-01-01 06:59:00,29.743000,-93.869500,214.0,0.0,9.6
629487,1,2019-01-01 08:53:00,29.743000,-93.869500,214.0,0.0,9.6
...,...,...,...,...,...,...,...
792015,176,2019-12-23 17:29:00,51.888150,1.672528,199.0,0.6,10.9
792017,176,2019-12-23 18:53:00,51.889468,1.674938,220.0,0.1,10.9
792018,176,2019-12-23 18:59:00,51.889498,1.674873,217.0,0.0,10.9
792019,176,2019-12-23 18:59:00,51.889498,1.674873,217.0,0.0,10.9


In [202]:
for i in range(len(voyages)):
  vessel = voyages['vessel'].iloc[i]
  end_date = voyages['end_date'].iloc[i]
  newtracking = tracking.loc[tracking['vessel'] == vessel]
  newtracking = newtracking.loc[newtracking['datetime'] == end_date]
  heading.append(newtracking.iloc[0][4])

voyages['heading'] = heading
voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id,time_spent,distance,efficiency,heading
0,1,2019-01-01 17:54:00,2019-01-16 22:59:00,138.0,102.0,15 days 05:05:00,7894.161113,518.949646,255.0
1,1,2019-01-17 19:26:00,2019-02-13 16:43:00,102.0,138.0,26 days 21:17:00,7894.161113,293.607253,213.0
2,1,2019-02-14 19:55:00,2019-03-01 10:51:00,138.0,33.0,14 days 14:56:00,7782.523134,532.239424,91.0
3,1,2019-03-02 04:54:00,2019-03-16 06:53:00,33.0,138.0,14 days 01:59:00,7782.523134,552.632443,214.0
4,1,2019-07-17 13:33:00,2019-08-12 23:43:00,138.0,72.0,26 days 10:10:00,11587.252161,438.518873,246.0
...,...,...,...,...,...,...,...,...,...
2055,176,2019-07-28 14:51:00,2019-08-01 08:49:00,64.0,136.0,3 days 17:58:00,2307.565977,615.578920,337.0
2056,176,2019-08-02 04:28:00,2019-08-14 08:55:00,136.0,64.0,12 days 04:27:00,2307.565977,189.371118,228.0
2057,176,2019-11-03 18:57:00,2019-11-20 02:30:00,64.0,56.0,16 days 07:33:00,3643.242907,223.312041,278.0
2058,176,2019-11-21 02:29:00,2019-12-02 21:48:00,56.0,136.0,11 days 19:19:00,2166.672542,183.540706,337.0


In [204]:
voyages.to_csv('dataset.csv')

In [208]:
corr = voyages.corr()
corr.style.background_gradient(cmap='coolwarm')

,vessel,begin_port_id,end_port_id,distance,efficiency,heading
vessel,1.000000,-0.033968,-0.034675,-0.426846,-0.056452,0.041264
begin_port_id,-0.033968,1.000000,-0.072412,0.039465,0.019398,-0.022829
end_port_id,-0.034675,-0.072412,1.000000,0.006420,-0.071339,0.121177
distance,-0.426846,0.039465,0.006420,1.000000,0.402289,-0.007597
efficiency,-0.056452,0.019398,-0.071339,0.402289,1.000000,-0.049250
heading,0.041264,-0.022829,0.121177,-0.007597,-0.049250,1.000000
